In [2]:
from PIL import Image
import numpy
import random
import math

apple = Image.open("banner.png")
array = numpy.array(apple)


for i in range(len(array)):
    for j in range(len(array[i])):
        if array[i][j][0] > 150:
            array[i][j] = [0,min(255,2 * array[i][j][1]),min(255,2 * array[i][j][2]), 255]
        array[i][j][0] += random.uniform(-0.1, 0.1)
        array[i][j][1] += random.uniform(-0.1, 0.1)
        array[i][j][2] += random.uniform(-0.1, 0.1)
        array[i][j][3] += random.uniform(-0.1, 0.1)

apple2 = Image.fromarray(array)
apple2.show()

FileNotFoundError: [Errno 2] No such file or directory: 'banner.png'

In [1]:
from PIL import Image
import numpy
from Controller import converter
from Controller import slam
import cv2
import math

from matplotlib import pyplot

leftImg, rightImg = numpy.split(numpy.array(Image.open("img.png")), 2, axis=1)

def show(img):
    Image.fromarray(img).show()

# show(leftImg)

# filteredLeftImg = converter.filter(leftImg)
# leftEdgesImage, gLeftIn, rLeftIn = cv2.split(filteredLeftImg)
leftEdgesIn, gLeftIn, rLeftIn = converter.filter(leftImg)
# show(leftBlurredIn)
show(leftEdgesIn)

# filteredRightImg = converter.filter(rightImg)
# rightEdgesImage, gRightIn, rRightIn = cv2.split(filteredRightImg)
rightEdgesIn, gRightIn, rRightIn = converter.filter(rightImg)
show(rightEdgesIn)

# distances, croppedLeft, rawHeightsLeft = converter.getDistances(leftBlurredIn, leftEdgesIn, rightBlurredIn, rightEdgesIn)
leftHeights, rightHeights = converter.getHeights(leftEdgesIn, rightEdgesIn)
# x = round(converter.focalLength / math.sqrt(3) + converter.imageWidth / 2)
# print(leftHeights[x], x)
# x = round(converter.imageWidth / 2 - converter.focalLength * math.sqrt(3))
# print(leftHeights[x], x)
# show(croppedLeft)
# for i in rawHeightsLeft:
#     croppedLeft[i][rawHeightsLeft[i]] = 125
# show(croppedLeft)
# image = []

slam.findStartingPosition(leftHeights, rightHeights)

print(slam.carX, slam.carY)

# print(rawHeightsLeft)

# # # print(heights)
# # # print(left)
# # print(distances)
# x = []
# y = []
# for i in distances:
#     # print(i)
#     # if i[2] < 200:
#     x.append(i[0])
#     y.append(i[1])
# # pyplot.xlim(-50, 50)
# # pyplot.ylim(-50, 50)
# x.append(0)
# y.append(0)
# pyplot.axes().set_aspect('equal')
# pyplot.scatter(x, y)
# for i in range(len(rawHeightsLeft)):
#     print(rawHeightsLeft[i])
#     print(distances[i])

# for i in distances:
#     image.append([i[2], i[2], i[2]])
# show(numpy.array(image))
# show(numpy.concatenate(([distances], [distances], [distances]), axis=0))
# leftBlobs, gBlobs = converter.getBlobs(rLeftIn, gLeftIn, rRightIn, gRightIn)

# landmarks = converter.getWallLandmarks(leftHeights, leftBlobs)


# corners = cv2.goodFeaturesToTrack(leftEdgesIn, 27, 0.5, 10)
# corners = numpy.int0(corners)
# print(corners)
# print(landmarks)
# for l in corners:
#     for i in range(-3, 3):
#         for j in range(-3, 3):
#             leftImg[l[0][1] + i][l[0][0] + j] = [255, 0, 0]
# for l in landmarks:
#     # leftEdgesIn[max(round(l[1]), 0)][l[0]] = 255
#     # leftEdgesIn[round(converter.imageHeight / 2 + 15 + leftHeights[l[0]]) + 2][l[0]] = 255
#     for i in range(-3, 3):
#         for j in range(-3, 3): 
#             if l[1] > 0:
#                 leftImg[round(converter.imageHeight / 2 + 15 + leftHeights[l[0]]) + j][l[0] + i] = [255, 0, 0]
#             else:
#                 leftImg[round(converter.imageHeight / 2 + 15 + leftHeights[l[0]]) + j][l[0] + i] = [0, 255, 0]

# show(leftEdgesIn)
# show(leftImg)
# outerWallLandmarks, innerWallLandmarks, rBlobLandmarks, gBlobLandmarks = converter.getLandmarks(distances, rBlobs, gBlobs)

# # for landmark in outerWallLandmarks:
# #     if landmark[2] < 30:
# #         print(landmark)
# print(outerWallLandmarks, innerWallLandmarks, rBlobLandmarks, gBlobLandmarks)

# print(converter.verticalFocalLength, converter.horizontalFocalLength)

10.5 49.0 42.0
-1 -1


In [2]:
import math

horizontalFov = 155
verticalFov = 115
imageWidth = 272
imageHeight = 154
focalLength = ((imageHeight / 2) / math.tan(math.pi * (verticalFov / 2) / 180))
wallHeight = 10
centerOffset = 10
dist = wallHeight * math.sqrt(focalLength**2 + (imageWidth / 2 - 29)**2) / 36
# return (dir * (3 + a[1] * dist), (10 + a[2] * dist), dist)

print(focalLength)
dist

49.05441008217698


32.69685768082119